In [2]:
import sys
sys.path.append('F:\Frederik\pfizer_mcrpc\paqc')

import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from paqc.utils import config_utils, utils
from paqc.connectors import csv

In [12]:
dict_config = config_utils.config_open(r'general_config_onc_EMR.yml')[1]

In [13]:
df = csv.read_csv(dict_config, 'F:\Projects\Pfizer_mCRPC\Data\Raw_data\Oncology_EMR\Oncology_EMR_cleaned_with_dummies.csv')

In [17]:
# Only numeric or datetime columns, good!
df.dtypes.unique()

array([dtype('int64'), dtype('<M8[ns]'), dtype('float64')], dtype=object)

In [18]:
df.head()

,patient_id,pn_flag,lookback_date,index_date,onc_latest_stage_date,onc_psa1,onc_psa_date1,onc_psa2,onc_psa_date2,onc_psa3,...,onc_latest_n_n2a,onc_latest_n_n2b,onc_latest_n_n2c,onc_latest_n_n3,onc_latest_n_n3a,onc_latest_m_m0,onc_latest_m_m1,onc_latest_m_m1a,onc_latest_m_m1b,onc_latest_m_m1c
0,1563563420,0,2012-05-14,2014-05-14,2006-11-01,15.6,2014-05-14,55.2,2014-02-19,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1364456765,0,2015-04-02,2017-04-01,NaT,NaN,NaT,NaN,NaT,NaN,...,0,0,0,0,0,0,0,0,0,0
2,523105747,0,2014-09-17,2016-09-16,2012-10-29,NaN,NaT,NaN,NaT,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1151578636,0,2014-10-04,2016-10-03,2016-10-06,NaN,NaT,NaN,NaT,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1427814564,0,2012-03-27,2014-03-27,NaT,NaN,NaT,NaN,NaT,NaN,...,0,0,0,0,0,0,0,0,0,0


In [22]:
ls_date_cols = utils.generate_list_columns(df, dict_config, ['date_cols'])
# Take index_date and lookback_date out of the list
ls_date_cols = ls_date_cols[2:]
len(ls_date_cols)

84

In [33]:
# Creating date difference variables
df_dd = pd.DataFrame()
for date_col in ls_date_cols:
    dd_col = date_col + '_index_dd'
    df_dd[dd_col] = (df['index_date'] - df[date_col]).dt.days

In [39]:
# Except for the onc_latest_stage_date, no column has dates after the index date
(df_dd < 0).iloc[:,1:].any().any()

False

In [43]:
df_no_date_cols = df.drop(columns = ls_date_cols)
df_no_date_cols = df_no_date_cols.drop(columns = ['lookback_date', 'index_date'])
df_no_date_cols.head()

,patient_id,pn_flag,onc_psa1,onc_psa2,onc_psa3,onc_psa4,onc_psa5,onc_psa6,onc_psa7,onc_psa8,...,onc_latest_n_n2a,onc_latest_n_n2b,onc_latest_n_n2c,onc_latest_n_n3,onc_latest_n_n3a,onc_latest_m_m0,onc_latest_m_m1,onc_latest_m_m1a,onc_latest_m_m1b,onc_latest_m_m1c
0,1563563420,0,15.6,55.2,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1364456765,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,523105747,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1151578636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1427814564,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [67]:
# Add the date difference columns to it
df_no_dates_with_dd = pd.concat([df_no_date_cols, df_dd], axis = 1)

In [68]:
df_no_dates_with_dd.head()

,patient_id,pn_flag,onc_psa1,onc_psa2,onc_psa3,onc_psa4,onc_psa5,onc_psa6,onc_psa7,onc_psa8,...,onc_total_testo_date18_index_dd,onc_free_testo_date1_index_dd,onc_free_testo_date2_index_dd,onc_free_testo_date3_index_dd,onc_free_testo_date4_index_dd,onc_tnm_date_index_dd,onc_latest_t_date_index_dd,onc_latest_n_date_index_dd,onc_latest_m_date_index_dd,onc_latest_gleason_date_index_dd
0,1563563420,0,15.6,55.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1364456765,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,523105747,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1151578636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1427814564,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
ls_columns = ['patient_id', 'pn_flag'] + sorted(df_no_dates_with_dd.columns.tolist()[2:], key=str)

In [70]:
df_no_dates_with_dd = df_no_dates_with_dd[ls_columns]

In [64]:
# Some columns are too spare to calculate bivariate stats out
ss_sparse = (~df_no_dates_with_dd.isnull()).sum() < 3
ls_sparse_cols = ss_sparse[ss_sparse].index.tolist()

In [73]:
ls_sparse_cols

['onc_free_testo4',
 'onc_free_testo_date4_index_dd',
 'onc_psa26',
 'onc_psa_date26_index_dd',
 'onc_total_testo11',
 'onc_total_testo12',
 'onc_total_testo13',
 'onc_total_testo14',
 'onc_total_testo15',
 'onc_total_testo16',
 'onc_total_testo17',
 'onc_total_testo18',
 'onc_total_testo_date11_index_dd',
 'onc_total_testo_date12_index_dd',
 'onc_total_testo_date13_index_dd',
 'onc_total_testo_date14_index_dd',
 'onc_total_testo_date15_index_dd',
 'onc_total_testo_date16_index_dd',
 'onc_total_testo_date17_index_dd',
 'onc_total_testo_date18_index_dd']

In [72]:
df_no_dates_with_dd_no_sparse = df_no_dates_with_dd.drop(columns=ls_sparse_cols)
df_no_dates_with_dd_no_sparse.head()

,patient_id,pn_flag,metac_site_bladder_cancer,metac_site_bone_connective_tissue_cancer,metac_site_brain_nervous_system_cancer,metac_site_breast_cancer,metac_site_colorectal_cancer_crc,metac_site_head_neck_cancer,metac_site_kidney_cancer,metac_site_liver_cancer,...,onc_total_testo_date4_index_dd,onc_total_testo_date5_index_dd,onc_total_testo_date6_index_dd,onc_total_testo_date7_index_dd,onc_total_testo_date8_index_dd,onc_total_testo_date9_index_dd,stage_i,stage_ii,stage_iii,stage_iv
0,1563563420,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
1,1364456765,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,523105747,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
3,1151578636,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
4,1427814564,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [74]:
df_no_dates_with_dd.to_feather('F:\Projects\Pfizer_mCRPC\Data\Raw_data\Oncology_EMR\Oncology_EMR_cleaned_with_dummies_with_dd.feather')
df_no_dates_with_dd.to_csv('F:\Projects\Pfizer_mCRPC\Data\Raw_data\Oncology_EMR\Oncology_EMR_cleaned_with_dummies_with_dd.csv')

df_no_dates_with_dd_no_sparse.to_feather('F:\Projects\Pfizer_mCRPC\Data\Raw_data\Oncology_EMR\Oncology_EMR_cleaned_with_dummies_with_dd_no_sparse.feather')